# Teste de hipóteses

Este caderno abordará **testes de hipóteses**, que medem se uma descoberta provavelmente ocorreu por acaso e não por causa de uma variável hipotética. Imagine testar um novo medicamento que supostamente reduz a duração de um resfriado. Se o tempo médio de recuperação com o medicamento for menor do que o tempo médio sem o medicamento, a diferença deve ser grande o suficiente para que seja improvável que tenha sido devido ao acaso. Pode muito bem ser que o medicamento não tenha tido efeito.

Vamos primeiro usar a intuição, explorando o componente mais crítico desse raciocínio: o valor-p.

## Intuição do valor P

Quando alguém diz que algo é estatisticamente significativo, o que isso significa? É uma expressão muito usada, mas poucas pessoas param para explicar como a significância estatística funciona em nível matemático. Para entender a significância estatística, podemos voltar à invenção do valor-p em 1925.

Um matemático chamado Ronald Fisher estava em uma festa. Uma de suas colegas, Muriel Bristol, afirmou ser capaz de detectar quando o chá era servido antes do leite simplesmente provando-o. Intrigado com a afirmação, Ronald montou um experimento na hora.

Ele preparou oito xícaras de chá. Quatro delas receberam o leite primeiro; as outras quatro receberam o chá primeiro. Ele então pediu que ela identificasse a ordem de cada uma. Surpreendentemente, ela identificou todas corretamente e, se estivesse apenas chutando, a probabilidade de isso acontecer por acaso é de 1 em 70, ou 0,01428571. Isso é o que chamamos de **valor-p**, a probabilidade de algo ter acontecido por acaso e não por causa de uma explicação hipotética.

svg image

Quando você tem um valor de p muito baixo (convencionalmente < 0,05), isso indica que o evento provavelmente não ocorreu por acaso. Portanto, somos inclinados a pensar que Muriel tem essa habilidade especial de detectar quando o chá foi servido antes do leite, porque se ela estivesse apenas chutando aleatoriamente, teria apenas 1,42% de chance de acertar.

Este exemplo não captura todas as nuances de um valor de p, mas fornece a essência dele. Vejamos como esse conceito se aplica a uma média amostral e à distribuição normal.

## Teste de duas caudas

Vamos trazer o conjunto de dados da lâmpada e calcular sua média, desvio padrão e contagem.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import norm
import numpy as np 

X = pd.read_csv("https://raw.githubusercontent.com/thomasnield/machine-learning-demo-data/master/distribution/lightbulb_data.csv") \
    .squeeze()
X

mean, std, n = X.mean(), X.std(), X.count()
print("MEAN: ", mean)
print("STD: ", std)
print("n: ", n)

Com base nesses dados, podemos inferir que há 95% de probabilidade de a lâmpada durar aproximadamente entre 571,3 horas e 773,1 horas, conforme mostrado abaixo.

In [ ]:
from matplotlib.patches import Polygon

fig, ax = plt.subplots()
x_range = np.arange(mean-std*3, mean+std*3, .01) 
ax.plot(x_range, norm.pdf(x_range, mean, std)) # curva de sino

# .95 area 
a,b = norm.ppf(.025, mean, std), norm.ppf(.975, mean, std)
ix = np.linspace(a, b)
iy =  norm.pdf(ix, mean, std)
verts = [(a, 0), *zip(ix, iy), (b, 0)]
poly = Polygon(verts, facecolor='0.9', edgecolor='0.5')
ax.add_patch(poly)

# adiciona rótulos de texto
plt.text(mean, .003, '.95', fontsize = 22, ha='center')
plt.text(a, norm.pdf(a,mean,std), round(a,2), fontsize = 16, ha='right', color='blue')
plt.text(b, norm.pdf(b,mean,std), round(b,2), fontsize = 16, ha='left', color='blue')


plt.show()

Podemos calcular que o intervalo central captura a área de 0,95 usando a função de densidade cumulativa inversa (a função `ppf()` no SciPy), conforme mostrado abaixo.

In [ ]:
from scipy.stats import norm

print(norm.ppf(.025, mean, std), norm.ppf(.975, mean, std))

Digamos que um engenheiro fez uma alteração no projeto e testou 31 lâmpadas com essa nova adaptação. Ele relata, com satisfação, que a média nesta nova amostra é 790. Agora, considere isto... será uma coincidência? Vamos comparar essa nova média com a nossa distribuição atual da vida útil das lâmpadas.

In [ ]:
from matplotlib.patches import Polygon

new_mean = 790

# gráfico de plotagem
fig, ax = plt.subplots()
x_range = np.arange(mean-std*3, mean+std*3, .01) 
ax.plot(x_range, norm.pdf(x_range, mean, std)) # curva de sino

# .95 area 
a,b = norm.ppf(.025, mean, std), norm.ppf(.975, mean, std)
ix = np.linspace(a, b)
iy =  norm.pdf(ix, mean, std)
verts = [(a, 0), *zip(ix, iy), (b, 0)]
poly = Polygon(verts, facecolor='0.9', edgecolor='0.5')
ax.add_patch(poly)

# adiciona rótulos de texto
plt.vlines(x = new_mean, ymin = 0, ymax = .005,
           colors = 'red',
           label = r"\bar{x}_{new}")

plt.text(mean, .003, '.95', fontsize = 22, ha='center')
plt.text(new_mean, .005, round(new_mean,2), fontsize = 16, ha='center', va='bottom', color='red')


plt.show()

Hmm... observe aqui que, se quisermos ter 95% de certeza de que a mudança de engenharia teve algum efeito, logicamente a média da nova amostra deve estar fora dessa faixa de 0,95 (devido ao teorema do limite central). Portanto, estamos inclinados a acreditar com 95% de certeza que a mudança de engenharia melhorou o projeto.

Vamos formalizar um pouco mais essas ideias. Afirmamos que a **hipótese nula ($ H_0 $)** é o status quo: a média é 672,2 e a mudança de engenharia não teve efeito. Mas a **hipótese alternativa ($ H_1 $)** é que a média *não* é 672,2 com o novo projeto de engenharia, e acreditamos que ela teve efeito suficiente para rejeitar a hipótese nula.

$$
H_0: \mu = 672,2
$$

$$
H_1: \mu \ne 672,2
$$

Como essa nova média de 790 está fora da faixa de confiança de 95%, a chamamos de **estatisticamente significativa**, o que significa que é improvável que seja coincidência o suficiente para que possamos rejeitar a hipótese nula e promover a hipótese alternativa.

Mas *quão improvável* é que teríamos observado 790 com o design atual da lâmpada? Podemos dizer, grosso modo, que está fora da faixa de "coincidência" de 95%, como mostrado acima, mas por quanto? É aqui que o **valor-p** entra novamente. Vamos dar uma olhada na área restante em cada cauda, ​​que será de $ 0,025 $.

In [ ]:
from matplotlib.patches import Polygon

new_mean = 790

# gráfico de plotagem
fig, ax = plt.subplots()
x_range = np.arange(mean-std*3, mean+std*3, .01) 
ax.plot(x_range, norm.pdf(x_range, mean, std)) # curva de sino

def plot_tail(a, b): 
    # áreas de valor p
    ix = np.linspace(a, b)
    iy = norm.pdf(ix, mean, std)
    verts = [(a, 0), *zip(ix, iy), (b, 0)]
    poly = Polygon(verts, facecolor='0.9', edgecolor='0.5')
    ax.add_patch(poly)

plot_tail(mean-std*3, norm.ppf(.025, mean, std))
plot_tail(norm.ppf(.975, mean, std), mean+std*3)

# adiciona rótulos de texto
plt.vlines(x = new_mean, ymin = norm.pdf(new_mean, mean, std), ymax = .005,
           colors = 'red',
           label = r"\bar{x}_{new}")

plt.text(555, .0001, '.025', fontsize = 12, ha='center')
plt.text(775, .0001, '.025', fontsize = 12, ha='left')
plt.text(new_mean, .005, round(new_mean,2), fontsize = 16, ha='center', va='bottom', color='red')


plt.show()

No entanto, este não é o nosso valor-p. O valor-p precisa capturar qualquer probabilidade que seja igual ou menor em ambos os lados onde observamos a nova média. Afinal, estamos tentando provar significância, e isso inclui qualquer coisa que seja igualmente ou menos provável de acontecer. Vamos visualizar apenas essa área em ambos os lados, que no total será o valor-p.

In [ ]:
from matplotlib.patches import Polygon

new_mean = 790

# gráfico de plotagem
fig, ax = plt.subplots()
x_range = np.arange(mean-std*3, mean+std*3, .01) 
ax.plot(x_range, norm.pdf(x_range, mean, std)) # curva de sino 

def plot_tail(a, b): 
    # áreas de valor p
    ix = np.linspace(a, b)
    iy = norm.pdf(ix, mean, std)
    verts = [(a, 0), *zip(ix, iy), (b, 0)]
    poly = Polygon(verts, facecolor='0.9', edgecolor='0.5')
    ax.add_patch(poly)

plot_tail(mean-std*3, norm.ppf(1.0 - norm.cdf(new_mean, mean, std), mean, std))
plot_tail(new_mean, mean+std*3)

# adicionar rótulos de texto
plt.vlines(x = new_mean, ymin = 0, ymax = .005,
           colors = 'red',
           label = r"\bar{x}_{new}")

one_side_p_value  = 1.0 - norm.cdf(new_mean, mean, std)
print(one_side_p_value*2)
plt.text(530, .0005, round(one_side_p_value, 3), fontsize = 12, ha='center')
plt.text(800, .0005, round(one_side_p_value, 3), fontsize = 12, ha='left')
plt.text(new_mean, .005, round(new_mean,2), fontsize = 16, ha='center', va='bottom', color='red')


plt.show()

Portanto, ficamos com um valor-p de aproximadamente $ 0,022 $, que é dividido por ambas as extremidades como $ 0,011 $. Embora tentar identificar os intervalos dessas extremidades exija alguma matemática dedutiva e um pouco de código, o cálculo do valor-p em si pode ser feito em duas linhas de código. Podemos simplesmente calcular a área da extremidade direita usando a FCD na nova média amostral (e subtrair de 1,0 para obter a área da extremidade direita, não da esquerda) e, em seguida, dobrá-la para aproveitar a simetria da extremidade esquerda.

In [ ]:
# área da cauda direita
p_value_right_tail = 1.0 -  norm.cdf(new_mean, mean, std)

# valor p de ambas as caudas (simétricas)
p_value = p_value_right_tail*2 

print(p_value) # 0.02212358425605565

Em termos práticos, como interpretamos esse valor-p? Dizemos que a vida útil da lâmpada reprojetada de 790 tem 2,2% de probabilidade de ter ocorrido por acaso (incluindo a probabilidade de eventos igualmente ou menos prováveis). Se nosso limite for de 95% de confiança, ou um nível de significância de 5%, $ 0,022 $ é menor que $ 0,05 $, então podemos rejeitar nossa hipótese nula e promover a hipótese alternativa de que nossa lâmpada reprojetada é uma melhoria.

Alternativamente, digamos que a nova lâmpada apenas melhorou a média da amostra para 750. Observe abaixo que isso não estaria na faixa "estatisticamente significativa", embora seja maior que a média atual de 672,2.

In [ ]:
from matplotlib.patches import Polygon

new_mean = 750

# gráfico de plotagem
fig, ax = plt.subplots()
x_range = np.arange(mean-std*3, mean+std*3, .01) 
ax.plot(x_range, norm.pdf(x_range, mean, std)) # curva de sino 

def plot_tail(a, b): 
    # áreas de valor p
    ix = np.linspace(a, b)
    iy = norm.pdf(ix, mean, std)
    verts = [(a, 0), *zip(ix, iy), (b, 0)]
    poly = Polygon(verts, facecolor='0.9', edgecolor='0.5')
    ax.add_patch(poly)

plot_tail(mean-std*3, norm.ppf(.025, mean, std))
plot_tail(norm.ppf(.975, mean, std), mean+std*3)

# adicionar rótulos de texto
plt.vlines(x = new_mean, ymin = 0, ymax = .005,
           colors = 'red',
           label = r"\bar{x}_{new}")

plt.text(555, .0001, '.025', fontsize = 12, ha='center')
plt.text(775, .0001, '.025', fontsize = 12, ha='left')
plt.text(new_mean, .005, round(new_mean,2), fontsize = 16, ha='center', va='bottom', color='red')


plt.show()

Com um limite de 95%, não podemos atribuir a esta lâmpada um desempenho significativamente melhor do que a atual, pois há uma boa chance de que tenha sido apenas por acaso e que ela tenha o mesmo desempenho que a lâmpada atual. Qual a probabilidade? Bem, novamente, esse é o valor-p e podemos calculá-lo. Descobrimos que é $ 0,13 $, e isso é muito maior do que $ 0,05 $. Portanto, não podemos rejeitar a hipótese nula.

In [ ]:
new_mean = 750 

# área da cauda direita
p_value_right_tail = 1.0 -  norm.cdf(new_mean, mean, std)

# valor p de ambas as caudas (simétricas)
p_value = p_value_right_tail*2 

print(p_value) # 0.13072525593787487

E só para garantir, aqui está a área do valor p visualizada para uma nova média amostral de 750.

In [ ]:
from matplotlib.patches import Polygon

new_mean = 750

# gráfico de plotagem
fig, ax = plt.subplots()
x_range = np.arange(mean-std*3, mean+std*3, .01) 
ax.plot(x_range, norm.pdf(x_range, mean, std)) # curva de sino

def plot_tail(a, b): 
    # áreas de valor p
    ix = np.linspace(a, b)
    iy = norm.pdf(ix, mean, std)
    verts = [(a, 0), *zip(ix, iy), (b, 0)]
    poly = Polygon(verts, facecolor='0.9', edgecolor='0.5')
    ax.add_patch(poly)

plot_tail(mean-std*3, norm.ppf(1.0 - norm.cdf(new_mean, mean, std), mean, std))
plot_tail(new_mean, mean+std*3)

# adicionar rótulos de texto
plt.vlines(x = new_mean, ymin = 0, ymax = .005,
           colors = 'red',
           label = r"\bar{x}_{new}")

one_side_p_value  = 1.0 - norm.cdf(new_mean, mean, std)
print(one_side_p_value*2)
plt.text(555, .0015, round(one_side_p_value, 3), fontsize = 12, ha='center')
plt.text(775, .0015, round(one_side_p_value, 3), fontsize = 12, ha='left')
plt.text(new_mean, .005, round(new_mean,2), fontsize = 16, ha='center', va='bottom', color='red')


plt.show()

## Teste unilateral

Embora testes bicaudais não sejam tão robustos, existe outra maneira de realizar testes de hipóteses por meio de **testes unicaudais**. Expressamos hipóteses nulas e alternativas como desigualdades. Seriam:

$$
H_0: \mu <= 672,2
$$

$$
H_1: \mu > 672,2
$$

Vamos visualizar o valor de p para testes unicaudais, como mostrado abaixo.

In [ ]:
from matplotlib.patches import Polygon

new_mean = 790

# gráfico de plotagem
fig, ax = plt.subplots()
x_range = np.arange(mean-std*3, mean+std*3, .01) 
ax.plot(x_range, norm.pdf(x_range, mean, std)) # curva de sino

def plot_tail(a, b): 
    # áreas de valor p
    ix = np.linspace(a, b)
    iy = norm.pdf(ix, mean, std)
    verts = [(a, 0), *zip(ix, iy), (b, 0)]
    poly = Polygon(verts, facecolor='0.9', edgecolor='0.5')
    ax.add_patch(poly)

plot_tail(new_mean, mean+std*3)

# adicionar rótulos de texto
plt.vlines(x = new_mean, ymin = 0, ymax = .005,
           colors = 'red',
           label = r"\bar{x}_{new}")

one_side_p_value  = 1.0 - norm.cdf(new_mean, mean, std)
plt.text(800, .0005, round(one_side_p_value, 3), fontsize = 12, ha='left')
plt.text(new_mean, .005, round(new_mean,2), fontsize = 16, ha='center', va='bottom', color='red')


plt.show()

Isso simplifica o cálculo do valor p, incluindo apenas a cauda que corresponde à hipótese alternativa.

In [ ]:
new_mean = 790 

# área da cauda direita
p_value = 1.0 -  norm.cdf(new_mean, mean, std)

print(p_value) # 0.011061792128027825

Como isso afeta a rejeição da hipótese nula? Pergunte a si mesmo: qual delas define um limite mais alto? Você notará que, mesmo quando nosso objetivo é mostrar que podemos ter diminuído/aumentado algo (a vida útil da lâmpada), reformular nossa hipótese para mostrar qualquer impacto (maior ou menor) cria um limite de significância mais alto. Se nosso limite de significância for um valor-p de 0,05 ou menos, nosso teste unicaudal teve um valor-p muito menor, 0,011, em oposição ao teste bicaudal, que foi aproximadamente o dobro daquele com valor-p de 0,022.

Por esse motivo, é preferível usar o teste bicaudal em vez do unicaudal em muitas situações. Ele não enviesa o teste em uma direção e também define um limite mais alto para a aceitação da hipótese alternativa.

## Lidando com amostras menores

Conforme discutido na seção anterior, se você tiver uma amostra menor (n<31), troque a distribuição normal pela distribuição T. Por exemplo, digamos que o novo projeto da lâmpada tenha apenas um tamanho de amostra de 5. Podemos calcular o valor-p bicaudal conforme mostrado abaixo.

In [ ]:
from scipy.stats import t

new_mean = 790 
n = 5

# área da cauda direita
p_value_right_tail = 1.0 -  t.cdf(new_mean, df=n-1, loc=mean, scale=std)

# valor p de ambas as caudas (simétricas)
p_value = p_value_right_tail*2 

print(p_value) # 0.08401989317568481

Nosso valor-p é muito maior: 0,084. Como isso não ultrapassa nosso limite de 0,05, não podemos rejeitar a hipótese nula.

## Exercício

Uma plataforma de jogos online está tentando aumentar o tempo de jogo de seus usuários com um novo sistema de recompensas. Normalmente, o jogador médio joga por 95 minutos, com um desvio padrão de 20 minutos. Após testar o novo sistema de recompensas com 100 jogadores selecionados aleatoriamente, eles descobriram que o tempo médio de jogo aumentou para 125 minutos. Complete o código abaixo usando o teste bicaudal para determinar o valor de p e determine se isso é estatisticamente significativo com 95% de confiança.

In [ ]:
from scipy.stats import norm 

mean = 95
std = 20
new_mean = 125

# área da cauda direita
p_value_right_tail = 1.0 -  ?

# valor p de ambas as caudas (simétricas)
p_value = ?

print(p_value) 

### RESPOSTA A BAIXO

|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
v 

O valor de p é 0,1336 e, como é maior que 0,05, isso não rejeita a hipótese nula. Isso significa que não podemos afirmar que o novo sistema de recompensas teve algum impacto no tempo de jogo dos jogadores.

In [ ]:
from scipy.stats import norm 

mean = 95
std = 20
new_mean = 125

# área da cauda direita
p_value_right_tail = 1.0 -  norm.cdf(new_mean, mean, std)

# valor p de ambas as caudas (simétricas)
p_value = p_value_right_tail*2 

print(p_value) # 0.13361440253771617